In [1]:
pip install fredapi

Note: you may need to restart the kernel to use updated packages.


In [3]:
# imports
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as DataReader
import seaborn as sns
from datetime import datetime
from fredapi import Fred

In [ ]:
# META
# Define the stock symbol and the time period
ticker_symbol = "META"  # META
start_date = "2020-01-01"  # Start date (YYYY-MM-DD)
end_date = "2024-08-31"  # End date (YYYY-MM-DD)

# Fetch the stock data
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Display the data
print(stock_data)

[*********************100%***********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-02  206.750000  209.789993  206.270004  209.779999  209.150269   
2020-01-03  207.210007  210.399994  206.949997  208.669998  208.043594   
2020-01-06  206.699997  212.779999  206.520004  212.600006  211.961823   
2020-01-07  212.820007  214.580002  211.750000  213.059998  212.420425   
2020-01-08  213.000000  216.240005  212.610001  215.220001  214.573944   
...                ...         ...         ...         ...         ...   
2024-08-26  527.599976  528.000000  514.950012  521.119995  520.623352   
2024-08-27  518.979980  524.010010  515.309998  519.099976  518.605225   
2024-08-28  517.669983  521.090027  512.450012  516.780029  516.287476   
2024-08-29  519.049988  527.200012  515.679993  518.219971  517.726074   
2024-08-30  521.349976  523.539978  515.200012  521.309998  520.813171   

              Volume  
Date          

In [ ]:
# 1. FAMA-FRENCH 5 FACTORS
# Get the Fama French 5 factor model (monthly data)
column_names = ["date", "Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"]
data_ff5 = pd.read_csv(
    "F-F_Research_Data_5_Factors_2x3_daily.csv",
    usecols=range(7),  # Only read the first 7 columns
    names=column_names,
    header=None,
    skiprows=4,
)
data_ff5["date"] = (
    data_ff5["date"].astype(str).str[0:4]
    + "-"
    + data_ff5["date"].astype(str).str[4:6]
    + "-"
    + data_ff5["date"].astype(str).str[6:8]
)

data_ff5["date"] = pd.to_datetime(data_ff5["date"], format="%Y-%m-%d")
data_ff5["date"] = data_ff5["date"].dt.date
df_ffs = data_ff5.set_index("date")

print(df_ffs.head())
print(df_ffs.shape)

            Mkt-RF   SMB   HML   RMW   CMA     RF
date                                             
1963-07-01   -0.67  0.02 -0.35  0.03  0.13  0.012
1963-07-02    0.79 -0.28  0.28 -0.08 -0.21  0.012
1963-07-03    0.63 -0.18 -0.10  0.13 -0.25  0.012
1963-07-05    0.40  0.09 -0.28  0.07 -0.30  0.012
1963-07-08   -0.63  0.07 -0.20 -0.27  0.06  0.012
(15417, 6)


In [ ]:
# 2. ADS INDEX
data_ads = pd.read_excel("ADS_Index_Most_Current_Vintage.xlsx")
# manually replace : into -
df_ads = data_ads.set_index("date")
df_ads.index = pd.to_datetime(df_ads.index, format="%Y:%m:%d")

In [ ]:
# 3. FRED DATA
fred = Fred(api_key="54dfdd8235a4e8e857eb4e5e4e77aed4")
varList = [
    "T10Y3M",
    "DGS10",
    "OBMMIJUMBO30YF",  # term premium 10yr-3mon, 30 yr mortgage jumbo loan
    "DEXUSEU",
    "DEXJPUS",
    "DEXUSUK",  # spot exchange rates to EUR, JPY, GBP
    "CBBTCUSD",
    "CBETHUSD",  # cryptocurrencies
    "T10YIE",
    "DCOILBRENTEU",  # breakeven inflation + brent oil price
    "VIXCLS",  # implied volatilities
    "DAAA",
    "DBAA",  # corporate bond yield
    "AMERIBOR",
    "T5YIE",
    "BAMLH0A0HYM2",
    "BAMLH0A0HYM2EY",
    "DGS1",
    "DCOILWTICO",
    "DHHNGSP",
]

SP500 = fred.get_series("SP500")
SP500.name = "SP500"
df_fred = SP500

# merge data series
for i in range(0, len(varList)):
    data = fred.get_series(varList[i])
    data.name = varList[i]
    df_fred = pd.merge(df_fred, data, left_index=True, right_index=True)

In [32]:
print("df_ffs index:", df_ffs.index)
print("df_ads index:", df_ads.index)
print("df_fred index:", df_fred.index)

df_ffs index: Index([1963-07-01, 1963-07-02, 1963-07-03, 1963-07-05, 1963-07-08, 1963-07-09,
       1963-07-10, 1963-07-11, 1963-07-12, 1963-07-15,
       ...
       2024-09-17, 2024-09-18, 2024-09-19, 2024-09-20, 2024-09-23, 2024-09-24,
       2024-09-25, 2024-09-26, 2024-09-27, 2024-09-30],
      dtype='object', name='date', length=15417)
df_ads index: DatetimeIndex(['1960-03-01', '1960-03-02', '1960-03-03', '1960-03-04',
               '1960-03-05', '1960-03-06', '1960-03-07', '1960-03-08',
               '1960-03-09', '1960-03-10',
               ...
               '2024-11-07', '2024-11-08', '2024-11-09', '2024-11-10',
               '2024-11-11', '2024-11-12', '2024-11-13', '2024-11-14',
               '2024-11-15', '2024-11-16'],
              dtype='datetime64[ns]', name='date', length=23637, freq=None)
df_fred index: DatetimeIndex(['2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06',
               '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12',
               '2017

In [ ]:
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)

df_ffs.index = pd.to_datetime(df_ffs.index)

mask = (df_ffs.index >= start_date) & (df_ffs.index <= end_date)
df_ffs = df_ffs.loc[mask]

DATA_part1 = pd.merge(df_ffs, df_ads, left_index=True, right_index=True)
DATA = pd.merge(DATA_part1, df_fred, left_index=True, right_index=True)
DATA.reset_index(inplace=True)
DATA = DATA.rename(columns={"index": "Date"})

DATA.to_csv("INFO6105_FeatureMart.csv", index=True)